In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from pyquery import PyQuery as pq
from bs4 import BeautifulSoup
import requests

In [119]:
#get wiki for all bachelor seasons
allseasons = requests.get("https://en.wikipedia.org/wiki/The_Bachelor_(U.S._TV_series)#Seasons")
#make beautiful soup element
soup = BeautifulSoup(allseasons.text, "html.parser")

#get the table cell that has links to each episode
seasons = soup.find("table", attrs={"class":"navbox"}).find("td", attrs={"class":"navbox-list navbox-odd hlist"})
seasons = seasons.find("div", attrs={"style":"padding:0em 0.25em"}).find("ul")

urls = []                           #list of links to season-specific page
seasonNums = []                     #list of seasons w/ wiki pages (no seasons 1-4 or 6-8)
seasonNum = 1                       #season number
for item in seasons.find_all("li"): #for each item in list of seasons
    if (seasonNum == 20):           #don't include season 20, b/c no contestants listed yet
        break
    season = item.find("a")         #get url tag
    if season is not None:          #if has url link, get url text
        urls.append("\"https://en.wikipedia.org" + season.get("href") + "\"")
        seasonNums.append(seasonNum) #add season number to list 
    seasonNum += 1
    
wikiPageText = []                   #init list of wiki site text, for all seasons
for url in urls:
    site = requests.get(url[1:-1])  #get web-site for that url
    soup = BeautifulSoup(site.text, "html.parser") #make BS element
    wikiPageText.append(soup)       #add web-site text to list

wikiPages = dict(zip(seasonNums, wikiPageText)) #key=season, val=Soup Elem(wiki page text)

In [120]:
# For each season in wiki, make list of dictionaries - one dictionary for each contestant.
# dictionary name = seasonsDict
#             key = season number
#           value = list of dictionaries for that season (one for each contestant)
#             
# For contestant dictionaries:
#            keys = name, age, hometown, occupation, elimination, season
#          values = associated values to fields, as scraped from wiki
#
# To test contestant dictionaries:
#         print seasonsDict[season][contestant][fieldname]
#    eg:  print seasonsDict[9][10]['name']  -- get name for season 9, contestant 10
#
# Note: Wiki does not have pages dedicated to Seasons 1-4, or 6-8. Also, Wiki does not list
# contestants for episode 20.  Those Seasons will be added to the dictionary later.

seasonsDict = dict()                #key = season num, val=list of contestant dictionaries
allContestants = dict()             #keys = name/age/etc, values = associated data

for sn in seasonNums:
    seasonPage = wikiPages[sn]      #get BS element for this season
    seasonPage = seasonPage.find("div", attrs={"id":"content"}).find("div", attrs={"id":"bodyContent"})
    seasonPage = seasonPage.find("div", attrs={"id":"mw-content-text"})
    seasonPage = seasonPage.find("table", attrs={"class":"wikitable sortable"})
    
    listOfContestantDicts = []          #list of dicts for each contestant
    
    numtr = 0                           #num rows (one per contestant)
    for tr in seasonPage.find_all("tr"):#for each contestant listed,
        if (numtr == 0):                #skip first row (column headers)
            numtr += 1
            continue

        contestantDict = dict()         #init new dict for contestant
        numtd = 0                       #column number
        for td in tr.find_all("td"):    #for each column of data,
            
            if (numtd == 0):
                name = str(td.contents)
                if ("<b>" in name):
                    td.find("b")
                    name = str(td.contents)[4:-5]
                if ("[u'" in name):                     #if in format "[u'name']",
                    name = name.encode('utf8')[3:-2]    #format to get 'name'
                if ("<span class" in name):
                    td.find("span", attrs={"class":"nowrap"})
                    tag = "<span class='nowrap'>"       #start tag before name
                    name = str(td.contents)[len(tag)+1:]#cut out start tag
                    end = name.index("<")               #get start point of end tag
                    name = name[:end]                   #cut out end tag
                    trashTag = "style=\"display:none;\">" #weird tag to cut from a name
                    if (trashTag in name):
                        name = name[(len(trashTag)+1):-1] 
                if ("<sup" in name):                    #if name has "name', <sup ...",
                    end2 = name.index("<sup")           #format to get name
                    name = name[:end2-3]
                contestantDict['name'] = name           #add name to dict

            if (numtd == 1):
                age = str(td.contents)
                if ("<b>" in age):
                    td.find("b")
                    age = str(td.contents)[4:-5]
                if ("[u'" in age):                      
                    age = age.encode('utf8')[3:5]       
                contestantDict['age'] = age
            if (numtd == 2):
                td.find("a")
                home = td.get("href")       
                home = td.get_text("title")
                if ("title" in home):                #format oddity in season 19, contest 1
                    indx = home.index("title")
                    home = home[:indx]
                contestantDict['hometown'] = home
            if (numtd == 3):
                job = str(td.contents)
                if ("<b>" in job):
                    td.find("b")
                    job = str(td.contents)[4:-5]
                if ("[u'" in job):                     
                    job = job.encode('utf8')[3:-2]      
                contestantDict['occupation'] = job   
            if (numtd == 4):
                elim = str(td.contents)
                if ("<b>" in elim):
                    td.find("b")
                    elim = str(td.contents)[4:-5]
                if ("[u'" in elim):                      
                    elim = elim.encode('utf8')[3:-2]      
                contestantDict['eliminated'] = elim
            numtd += 1
        numtr += 1
        contestantDict['season'] = sn   #include season num in dict
        
        listOfContestantDicts.append(contestantDict) #add dict to list of dicts
        
    seasonsDict[sn] = listOfContestantDicts  #key = season num, val=list of contestant dicts

#no seasons: 1-4, 6-8, 20
#to test: print seasonsDict [season][contestant][fieldname], eg:
#print seasonsDict[9][10]['name']

In [ ]:
#get data for seasons 1-4 and 6-8
#possible source:  http://abc.go.com/primetime/bachelor/index?pn=photos#t=31304

#good source: http://www.realitywanted.com/shows/the-bachelor/season-8-paris

In [121]:
# Enter data for Season 6. 
#
# 1) Source:(looks different in Safari versus Chrome) http://www.realitytvworld.com/#$$nxt6je&&9hhYJIraEeWixQqIPWP/qw$$
# 2) Alt. Source: http://www.realitytvworld.com/news/abc-releases-identities-of-sixth-bachelor-edition-bachelorettes-2880.php")
# 3) FYI, source without occupations, but with pictures: 
# (http://community.realitytvworld.com/cgi-sys/cgiwrap/rtvw2/community/dcboard.cgi?az=printer_format&om=894&forum=DCForumID42)
# 4) Source of winner name: http://www.realitywanted.com/shows/the-bachelor/season-6
# 5) Mary Delgado source: http://www.sptimes.com/2003/09/26/Tampabay/No_wedding_bells__jus.shtml

# Despite hours of trying, I could not get the article text from this site.
# So, I hand-entered the data.
# If I have time, I will come back to this.
# Below are initial notes:
# seasonSix = requests.get("url")
# six = BeautifulSoup(seasonSix.text, "html.parser") 
# six = season6.find("body") 
# six = six.find("table", attrs={"class":"header"})

season6 = ["Abby, a 29-year-old acrobat who currently resides in Henderson, NV", 
"Alma Rubenstein, a 35-year-old cafe owner who currently resides in Astoria, OR",
"Amanda, a 27-year-old cosmetics buyer who currently resides in New York, NY", 
"Amy, a 27-year-old marketing consultant who currently resides in San Diego, CA", 
"Andrea, a 33-year-old dental hygienist who currently resides in Denver, CO", 
"Ashley, a 31-year-old teacher who currently resides in Santa Barbara, CA", 
"Carolyn, a 36-year-old financial advisor who currently resides in Tulsa, OK", 
"Cheresse, a 31-year-old advertising director who currently resides in St. Louis, MO", 
"Cynthia, a 37-year-old charity foundations director who currently resides in Hermosa Beach, CA", 
"Elizabeth, a 28-year-old in pharmaceutical sales who currently resides in Chicago, IL", 
"Jayne, a 37-year-old dog groomer, who currently resides in Key Largo, FL",
"Jennifer, a 31-year-old account executive who currently resides in Seattle, WA", 
"Kelly, a 34-year-old actress who currently resides in Beverly Hills, CA", 
"Kerry, a 31-year-old nurse who currently resides in San Francisco, CA", 
"Kristie, a 32-year-old bar owner who currently resides in Windsor, Canada", 
"Kristin, a 27-year-old office manager who currently resides in Pensacola, FL", 
"Krysta, a 28-year-old financial analyst who currently resides in Oklahoma City, OK", 
"Leina, a 28-year-old advertising associate who currently resides in Chula Vista, CA", 
"Lisa, a 33-year-old teacher who currently resides in West Palm Beach, FL", 
"Melinda, a 39-year-old photographer who currently resides in Nashville, TN", 
"Natalie, a 34-year-old in retail sales who currently resides in Santa Monica, CA", 
"Nicole, a 28-year-old executive recruiter who currently resides in Libertyville, IL", 
"Susie, a 32-year-old insurance broker who currently resides in Hollywood, CA", 
"Tanya, a 31-year-old teacher who currently resides in Plano, Texas", 
"Wende, a 28-year-old model who currently resides in Austin, Texas",
"Mary Delgado, a 35-year-old real estate agent who currently resides in Tampa Bay, FL"]

for s6 in season6:
    firstComma = s6.index(',')                      #parse string
    startAge = s6.index(" a ")
    jobTag6 = "year-old "     
    startJob6 = s6.index(jobTag6)
    homeTag6 = "currently resides in "
    startHome = s6.index(homeTag6)
    
    contestantDict['name'] = s6[:firstComma]        #put field data into dictionary
    contestantDict['age'] = s6[startAge+3:startAge+5]
    contestantDict['occupation'] = s6[startJob6 + len(jobTag6):s6.index("who")-1]
    contestantDict['hometown'] = s6[startHome + len(homeTag6):]
    contestantDict['season'] = 6
    
    if ("Delgado" in s6):
        contestantDict['elimination'] = "Winner"
    
    listOfContestantDicts.append(contestantDict)   #add dict to list of dicts

seasonsDict[6] = listOfContestantDicts  #key = season num, val=list of contestant dicts

Abby
29
acrobat
Henderson, NV


In [218]:
# Get data for Season 8, add to dictionary

def getSeason8(season8Site):
    
    #get site with Bachelor Season 8 Contestants
    seasonEight = requests.get(season8Site)
    #make beautiful soup element
    season8= BeautifulSoup(seasonEight.text, "html.parser")

    #get the table cell that has links to each episode
    eight = season8.find("body", attrs={"id":"imagegalleryIndexPage"})
    eight = eight.find("main", attrs={"id":"main"})
    eight = eight.find("div", attrs={"class":"container"})
    eight = eight.find_all("div", attrs={"class":"row"})[1]
    eight = eight.find("div", attrs={"class":"col col-11"}).find("div", attrs={"class":"row"})
    eight = eight.find("div", attrs={"class":"col col-8"})
    eight = eight.find("div", attrs={"class":"content widget gallery-index-content"})
    eight = eight.find("ul")


    urls8 = []                       #list of urls for season 8 contestant pages
    for item in eight.find_all("li"):#for each contestant in list of season 8 contestants
        url8 = item.find("a")       #get url tag
        if url8 is not None:         #if has url link, get url 
            urls8.append("\"http://realitytv.about.com" + url8.get("href") + "\"")

    cont8Sites = []                  #list of soup objects for season 8 contestant sites
    for link in urls8:
        site8 = requests.get(link[1:-1]) 
        soup8 = BeautifulSoup(site8.text, "html.parser") #get soup element
        cont8Sites.append(soup8)     #add soup element to list

    for cont8 in cont8Sites:
        c8 = cont8.find("body", attrs={"id":"imagegalleryPage"})
        c8 = c8.find("main", attrs={"class":"slab"})
        c8 = c8.find("div", attrs={"class":"container"})
        c8 = c8.find_all("div", attrs={"class":"row"})[1]
        c8 = c8.find("div", attrs={"class":"col col-11"})
        c8 = c8.find("div", attrs={"id":"contentIntro"})
        c8 = c8.find("div", attrs={"class":"row"})
        c8 = c8.find("div", attrs={"class":"col col-6"})
        c8 = c8.find("div", attrs={"class":"muted subheading"}).getText()
    
        #get name
        firstComma = c8.index(',')
        contestantDict['name'] = c8[:firstComma]   #add name to dict
    
        #get age
        substrC8 = c8[firstComma+2:]
        secondComma = substrC8.index(',')
        contestantDict['age'] = substrC8[:secondComma]
            
        #get job
        if ("is a" in c8):
            jobtag = "is a "
        if ("is an" in c8):
            jobtag = "is an "
        jobIndex = c8.index(jobtag)  
        contestantDict['occupation'] = c8[(jobIndex+len(jobtag)):(c8.index("who"))]   #add name to dict

        #get hometown
        hometag = "resides in "
        homeIndex = c8.index(hometag)
        contestantDict['hometown'] = c8[(homeIndex+len(hometag)):-1]
        
        if ("Sarah Stone" in name):                 #hard-code winner
            contestant['eliminatied'] = "Winner"
    
        #add season
        contestantDict['season'] = 8
        
        #add dict to list of dicts
        listOfContestantDicts.append(contestantDict)   

seasonsDict[8] = listOfContestantDicts  #key = season num, val=list of contestant dicts
    
    
#call 'getSeason8' to scrape data from both sites that have season 8 data
getSeason8("http://realitytv.about.com/od/thebachelor8/ig/Ladies-of-The-Bachelor--Paris/")
getSeason8("http://realitytv.about.com/od/thebachelor8/ig/Ladies-of-The-Bachelor--Paris/index.01.htm#step-heading")

29
29
